In [152]:
import csv
import nltk
import re
from nltk.corpus import stopwords
import pandas as pd
import numpy as np
import matplotlib
from sklearn.ensemble import RandomForestClassifier
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectFromModel
from sklearn import preprocessing
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [3]:
x=pd.read_csv('Womens Clothing E-Commerce Reviews.csv')

In [45]:
# No of unique classes  =  21
# No of unique divisions  =  4
# No of unique departments  =  7
# No of unique ratings  =  5
# No of unique ages  =  77
# No of unique positive feedback  =  82

In [120]:
review_data = x['Review Text']
review_set = list(review_data)
id_data = x['Clothing ID']
rating_data=x['Rating']
for i in range(len(review_set)):
    if type(review_set[i]) is str:
        review_set[i]= review_set[i].lower()

In [112]:
#Reading Reviews
train_length = (len(review_data)*2)//3
test_length = len(review_data) - train_length
reviews_train = [review_set[i] for i in range(0,train_length)] #2/3 of data for training
reviews_test = [review_set[i] for i in range(train_length,test_length)] #1/3 of data for testing

In [148]:
REPLACE_NO_SPACE = re.compile("(\.)|(\;)|(\:)|(\!)|(\')|(\?)|(\,)|(\")|(\()|(\))|(\[)|(\])")
REPLACE_WITH_SPACE = re.compile("(<br\s*/><br\s*/>)|(\-)|(\/)")

def preprocess_reviews(reviews):
    reviews = [REPLACE_NO_SPACE.sub("", str(reviews[i])) for i in range(len(reviews))]
    reviews = [REPLACE_WITH_SPACE.sub(" ", str(reviews[i])) for i in range(len(reviews))]
    return reviews

reviews_train_clean = preprocess_reviews(reviews_train)
reviews_test_clean = preprocess_reviews(reviews_test)

In [162]:
# Create one very large matrix with one column for every unique word in your corpus (where the corpus is all clean reviews
# in our case). Then we transform each review into one row containing 0s and 1s, where 1 means that the word in the 
# corpus corresponding to that column appears in that review.

cv = CountVectorizer(binary=True)
cv.fit(reviews_train_clean)
X = cv.transform(reviews_train_clean)
X_test = cv.transform(reviews_test_clean)

In [174]:
target = [1 if i < train_length//2 else 0 for i in range(train_length)]

X_train, X_val, y_train, y_val = train_test_split(X, target, train_size = 0.75,test_size = 0.25)

for c in [0.01, 0.05, 0.25, 0.5, 1]:
    
    lr = LogisticRegression(C=c)
    lr.fit(X_train, y_train)
    print ("Accuracy for C=%s: %s" %(c, accuracy_score(y_val, lr.predict(X_val))))

Accuracy for C=0.01: 0.5139208173690932
Accuracy for C=0.05: 0.5090676883780332
Accuracy for C=0.25: 0.5236270753512133
Accuracy for C=0.5: 0.5233716475095785
Accuracy for C=1: 0.5228607918263091


In [172]:
final_model = LogisticRegression(C=1)
final_model.fit(X, target)
print ("Final Accuracy: %s" % accuracy_score(target, final_model.predict(X_val)))

ValueError: Found input variables with inconsistent numbers of samples: [15657, 3915]

In [169]:
feature_to_coef = {
    word: coef for word, coef in zip(
        cv.get_feature_names(), final_model.coef_[0]
    )
}
for best_positive in sorted(feature_to_coef.items(), key=lambda x: x[1], reverse=True)[:5]:
    print (best_positive)
    
for best_negative in sorted(
feature_to_coef.items(),
    key=lambda x: x[1])[:5]:
    print (best_negative)
    

('unfortunate', 1.4382214372914595)
('laser', 1.387705442010196)
('halter', 1.3270011330607936)
('yikes', 1.326541310359293)
('downs', 1.3152623670870889)
('vegan', -1.9252936626163464)
('brocade', -1.6212200964733696)
('losing', -1.5309302679673897)
('june', -1.525312785750655)
('frames', -1.5231170135984804)
